## 卷积神经网络

In [1]:
import tensorflow as tf
import os
import pickle
import numpy as np

CIFAR_DIR = "./data/cifar-10-batches-py"
print(os.listdir(CIFAR_DIR))

['batches.meta', 'data_batch_1', 'data_batch_2', 'data_batch_3', 'data_batch_4', 'data_batch_5', 'readme.html', 'test_batch']


In [2]:
def load_data(filename):
    with open(filename, 'rb') as f:
        data = pickle.load(f, encoding='latin1')
        return data['data'], data['labels']

class CifarData:
    # 对于训练集需要每次选取时对数据进行shuffle，因为在训练时要进行w参数的更新，shuffle可以减少数据之间的相互依赖关系
    # 对于测试数据集只计算loss和accuracy，无需shuffle
    def __init__(self, filenames, need_shuffle):
        all_data = []
        all_labels = []
        for filename in filenames:
            data, labels = load_data(filename)
            all_data.append(data)
            all_labels.append(labels)
        self._data = np.vstack(all_data) # 这里将保存好的数据合成一个矩阵
        # 这里没有进行归一化的分类效果比较差，是因为原像素的值太大，但是预测的值都在0，1之间，
        # sigmoid函数在x取值较大时会出现梯度弥散的现象，导致学习不到参数
        self._data = self._data / 127.5 - 1 # 对图像进行归一化   
        self._labels = np.hstack(all_labels)
        self._num_examples = self._data.shape[0]
        self._need_shuffle = need_shuffle
        self._indicator = 0 # 数据集遍历位置指示器
        if self._need_shuffle:
            self._shuffle_data()
            
    def _shuffle_data(self):
        # 生成一个制定范围的shuffle整数序列
        p = np.random.permutation(self._num_examples)
        self._data = self._data[p]
        self._labels = self._labels[p]
    
    def next_batch(self, batch_size):
        """return batch size examples as a batch"""
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data() # 如果end已经大于data size的话说明数据已经遍历完，所以将指示器归零
                self._indicator = 0
                end_indicator = batch_size
            else:
                raise Exception('have no more examples')
        if end_indicator > self._num_examples:
            raise Exception("batch size is larger than all examples")
        batch_data = self._data[self._indicator: end_indicator]
        batch_labels = self._labels[self._indicator:end_indicator]
        self._indicator = end_indicator
        return batch_data, batch_labels
train_filenames = [os.path.join(CIFAR_DIR, 'data_batch_%d' % i) for i in range(1, 6)]
test_filenames = [os.path.join(CIFAR_DIR, 'test_batch')]
train_data = CifarData(train_filenames, True)
test_data = CifarData(test_filenames, False)

batch_data, batch_labels = train_data.next_batch(10)


In [3]:
x = tf.placeholder(tf.float32, [None, 3072])
y = tf.placeholder(tf.int64, [None])

# 十分类
# （3072，10)
# 全连接层封装函数
hidden1 = tf.layers.dense(x, 100, activation=tf.nn.relu)
hidden2 = tf.layers.dense(hidden1, 100, activation=tf.nn.relu)
hidden3 = tf.layers.dense(hidden2, 50, activation=tf.nn.relu)

y_ = tf.layers.dense(hidden3, 10)


# cross entropy
loss = tf.losses.sparse_softmax_cross_entropy(labels=y, logits=y_)
# y_ -> softmax
# y -> onehot
# loss = ylogy_



# y_是一个矩阵，第一个维度是所有样本，第二个维度是对应的每个样本在每一个类别上的预测值，获取预测值最大的类别的index
predict = tf.argmax(y_, 1)
# [1,0,1,0...]
correct_prediction = tf.equal(predict, y)
# 为了计算均值，这里将预测的int值转为float类型
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float64))

with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)


In [4]:
init = tf.global_variables_initializer()
batch_size = 20
train_steps = 10000
test_steps = 100

with tf.Session() as sess:
    sess.run(init)
    for i in range(train_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)
        loss_val, acc_val,  _ = sess.run(
            [loss, accuracy, train_op], 
            feed_dict={x: batch_data, y: batch_labels})
        if (i+1) % 500 == 0:
            print('[Train] step: %d, loss: %4.5f, acc: %4.5f'\
                  %(i+1, loss_val, acc_val))
        if (i+1) % 5000 == 0:
            test_data = CifarData(test_filenames, False)
            all_test_acc_val = []
            for j in range(test_steps):
                test_data_batch, test_labels_batch \
                =test_data.next_batch(batch_size)
                test_acc_val = sess.run([accuracy], feed_dict={x:test_data_batch, y:test_labels_batch})
                all_test_acc_val.append(test_acc_val)
            test_acc = np.mean(all_test_acc_val)
            print("[Test] step: %d, acc: %4.5f" % (i+1, test_acc))

[Train] step: 500, loss: 1.67296, acc: 0.30000
[Train] step: 1000, loss: 1.85763, acc: 0.45000
[Train] step: 1500, loss: 1.75073, acc: 0.35000
[Train] step: 2000, loss: 1.73998, acc: 0.50000
[Train] step: 2500, loss: 1.90858, acc: 0.25000
[Train] step: 3000, loss: 1.42281, acc: 0.55000
[Train] step: 3500, loss: 2.01301, acc: 0.45000
[Train] step: 4000, loss: 1.55739, acc: 0.40000
[Train] step: 4500, loss: 1.03378, acc: 0.60000
[Train] step: 5000, loss: 1.59755, acc: 0.45000
[Test] step: 5000, acc: 0.49150
[Train] step: 5500, loss: 1.12241, acc: 0.60000
[Train] step: 6000, loss: 1.39427, acc: 0.55000
[Train] step: 6500, loss: 1.46522, acc: 0.50000
[Train] step: 7000, loss: 1.10342, acc: 0.65000
[Train] step: 7500, loss: 1.13335, acc: 0.50000
[Train] step: 8000, loss: 1.01019, acc: 0.70000
[Train] step: 8500, loss: 1.35927, acc: 0.60000
[Train] step: 9000, loss: 1.52345, acc: 0.40000
[Train] step: 9500, loss: 1.36622, acc: 0.45000
[Train] step: 10000, loss: 1.64157, acc: 0.35000
[Test] s